# Visualization with lwcharts

This notebook demonstrates how to visualize charts and patterns using numta's integration with lwcharts.

## Installation

```bash
pip install "numta[viz]"
```

This installs lwcharts along with pandas support.

In [ ]:
import numpy as np
import pandas as pd
import numta

# Check if lwcharts is available
try:
    from numta.viz import (
        plot_chart, plot_ohlc, plot_line, plot_with_indicators,
        plot_pattern, plot_harmonic, StreamingChart
    )
    LWCHARTS_AVAILABLE = True
    print("lwcharts is available")
except ImportError:
    LWCHARTS_AVAILABLE = False
    print("lwcharts not installed. Run: pip install lwcharts")

## Creating Sample Data

In [ ]:
np.random.seed(42)
n = 200

# Generate realistic OHLCV data
dates = pd.date_range('2024-01-01', periods=n, freq='D')
base = 100 + np.cumsum(np.random.randn(n) * 1.5)

df = pd.DataFrame({
    'open': base + np.random.randn(n) * 0.5,
    'high': base + np.abs(np.random.randn(n)) * 2,
    'low': base - np.abs(np.random.randn(n)) * 2,
    'close': base,
    'volume': np.random.randint(1000, 10000, n).astype(float)
}, index=dates)

# Ensure high/low are correct
df['high'] = df[['open', 'high', 'close']].max(axis=1)
df['low'] = df[['open', 'low', 'close']].min(axis=1)

print(df.head())

## Basic Charts

### Candlestick Chart

In [ ]:
if LWCHARTS_AVAILABLE:
    # Basic candlestick chart
    chart = plot_ohlc(df, volume=True)
    chart.load()

### Line Chart

In [ ]:
if LWCHARTS_AVAILABLE:
    # Line chart from close prices
    chart = plot_line(df['close'], title='Close Price')
    chart.load()

## Adding Indicators

The `plot_with_indicators` function overlays indicators on the chart.

In [ ]:
# Calculate some indicators
df.ta.sma(timeperiod=20, append=True)
df.ta.ema(timeperiod=10, append=True)

print(df.columns.tolist())

In [ ]:
if LWCHARTS_AVAILABLE:
    # Chart with indicators overlay
    indicators = {
        'SMA_20': df['SMA_20'],
        'EMA_10': df['EMA_10']
    }
    
    chart = plot_with_indicators(df, indicators=indicators, volume=True)
    chart.load()

### Using the .ta Accessor for Plotting

In [ ]:
if LWCHARTS_AVAILABLE:
    # Direct plotting from DataFrame
    chart = df.ta.plot(indicators={'SMA_20': df.ta.sma(20), 'EMA_10': df.ta.ema(10)})
    chart.load()

## Pattern Visualization

Visualize detected patterns on charts.

In [ ]:
# Find chart patterns
patterns = df.ta.find_patterns(pattern_type='all', order=5)
print(f"Patterns found: {len(patterns)}")

In [ ]:
if LWCHARTS_AVAILABLE and len(patterns) > 0:
    # Plot with patterns
    chart = plot_pattern(df, patterns, show_trendlines=True)
    chart.load()

In [ ]:
if LWCHARTS_AVAILABLE:
    # Alternative: use the .ta accessor
    patterns = df.ta.find_patterns()
    chart = df.ta.plot(patterns=patterns)
    chart.load()

### Harmonic Pattern Visualization

In [ ]:
# Find harmonic patterns
harmonics = df.ta.find_harmonic_patterns()
print(f"Harmonic patterns found: {len(harmonics)}")

In [ ]:
if LWCHARTS_AVAILABLE and len(harmonics) > 0:
    # Plot with harmonic patterns and PRZ zones
    chart = plot_harmonic(df, harmonics, show_prz=True)
    chart.load()

## Streaming Charts

For real-time visualization, use `StreamingChart`.

In [ ]:
if LWCHARTS_AVAILABLE:
    from numta.streaming import StreamingSMA
    
    # Create streaming chart
    streaming_chart = StreamingChart(max_points=500, title="Real-Time Chart")
    sma = StreamingSMA(timeperiod=20)
    
    # Simulate streaming data
    for i in range(50):
        open_ = 100 + np.random.randn() * 0.5
        close = 100 + np.random.randn() * 0.5
        high = max(open_, close) + abs(np.random.randn()) * 0.3
        low = min(open_, close) - abs(np.random.randn()) * 0.3
        volume = 1000 + np.random.randint(0, 500)
        
        # Add bar to chart
        streaming_chart.add_bar(i, open_, high, low, close, volume)
        
        # Update indicator
        sma_value = sma.update(close)
        if sma_value is not None:
            streaming_chart.add_indicator('SMA_20', i, sma_value, color='blue')
    
    # Display
    streaming_chart.show()

## Customization Options

lwcharts provides many customization options.

In [ ]:
if LWCHARTS_AVAILABLE:
    # Customized chart
    chart = plot_ohlc(
        df,
        volume=True,
        title='Custom Chart',
        width=900,
        height=500
    )
    chart.load()

### Chart Themes

In [ ]:
if LWCHARTS_AVAILABLE:
    # Dark theme chart
    chart = plot_chart(
        df,
        chart_type='candlestick',
        theme='dark'
    )
    chart.load()

## Multiple Panes

Add indicators to separate panes below the main chart.

In [ ]:
# Calculate RSI and MACD
df.ta.rsi(timeperiod=14, append=True)
df.ta.macd(append=True)

print(df.columns.tolist())

In [ ]:
if LWCHARTS_AVAILABLE:
    # Chart with multiple panes
    from numta.viz import MultiPaneChart
    
    # Create multi-pane chart (if supported)
    try:
        chart = MultiPaneChart()
        chart.add_candlestick_pane(df)
        chart.add_indicator_pane(df['RSI_14'], name='RSI')
        chart.add_indicator_pane(df['MACD_12_26_9'], name='MACD')
        chart.load()
    except (ImportError, AttributeError):
        print("MultiPaneChart requires additional configuration")

## Summary

numta's visualization module provides:

- **Basic charts**: Candlestick, OHLC, line charts
- **Indicator overlays**: SMA, EMA, Bollinger Bands, etc.
- **Pattern visualization**: Chart patterns with trendlines
- **Harmonic patterns**: PRZ zones and Fibonacci levels
- **Streaming charts**: Real-time updates
- **Customization**: Themes, colors, sizes

All visualization features work in Jupyter notebooks with interactive lwcharts.

## Next Steps

- See `08_performance_optimization.ipynb` for performance tips